In [10]:
# REPRESENTACIÓN, NORMALIZACIÓN Y LIMPIEZA DE DATOS 

import pandas as pd


# Función para limpiar y convertir valores de mercado
def clean_market_value(column):
    # Eliminar el símbolo € si existe
    column_cleaned = column.str.replace('€', '', regex=False)
    
    # Convertir millones ('m') a su valor numérico real
    column_cleaned = column_cleaned.apply(lambda x: float(x.replace('m', '')) * 1_000_000 if isinstance(x, str) and 'm' in x else x)
    
    # Convertir miles ('k') a su valor numérico real
    column_cleaned = column_cleaned.apply(lambda x: float(x.replace('k', '')) * 1_000 if isinstance(x, str) and 'k' in x else x)
    
    # Convertir finalmente a numérico
    return pd.to_numeric(column_cleaned, errors='coerce')

# Leer todas las hojas del archivo Excel en un diccionario de DataFrames
sheets_dict = pd.read_excel(r'C:\Users\User\Downloads\BD_TotalMarketValue.xlsx', sheet_name=None, header=1)

# Crear un nuevo diccionario para almacenar las hojas modificadas
cleaned_sheets_dict = {}

# Iterar sobre cada hoja y realizar las operaciones necesarias
for sheet_name, df in sheets_dict.items():
    
    # Normalizar el nombre de las columnas
    df = df.rename(columns=lambda x: x.strip().lower())  # Elimina espacios en blanco y convierte a minúsculas

    # Eliminar la columna "unnamed: 0" si existe
    if 'unnamed: 0' in df.columns:
        df = df.drop(columns=['unnamed: 0'])

    # Limpiar las columnas 'total market value' y 'market value'
    if 'total market value' in df.columns:
        df['total market value'] = clean_market_value(df['total market value'])
    if 'market value' in df.columns:
        df['market value'] = clean_market_value(df['market value'])

    # Método fillna para imputar los valores NaN
    df.fillna({'total market value': df['total market value'].median(), 'market value': df['market value'].median()}, inplace=True)

    # Guardar el DataFrame modificado en el nuevo diccionario
    cleaned_sheets_dict[sheet_name] = df
    
    print(f"Hoja procesada: {sheet_name}")
    print(df)  
    print(df.isna().sum())  
    print(df.head())
    print(df.shape)
    print(df.info())   
    

    


Hoja procesada: Año_2009
                          club  squad   age  foreigners  market value  \
0               Deportivo Cali     43  22.2           7      148000.0   
1            Atlético Nacional     37  23.6           6      161000.0   
2                    Junior FC     35  25.5           3      163000.0   
3               Millonarios FC     38  23.4           5       92000.0   
4                  Once Caldas     37  23.8           3       92000.0   
5            Deportivo Pereira     45  23.1           4       67000.0   
6                  Envigado FC     38  22.6           1       62000.0   
7             Cúcuta Deportivo     40  24.1           3       53000.0   
8           CD América de Cali     55  23.9           7       36000.0   
9       Independiente Medellín     47  21.0           6       27000.0   
10    CD La Equidad Seguros SA     40  23.3           6       30000.0   
11      Independiente Santa Fe     38  22.9           7       12000.0   
12             Boyacá Chic

In [ ]:
# ANÁLISIS EXPLORATORIO DE DATOS EDA


import matplotlib.pyplot as plt
import seaborn as sns

# Función para realizar el análisis en cada hoja
def analyze_sheet(df, year):
    print(f'Análisis del Año {year}:')
    
    # Verificar si la columna 'total market value' existe
    if 'total market value' not in df.columns:
        print(f"Error: La columna 'total market value' no se encuentra en la hoja para el Año {year}")
        return  # Saltar el análisis si la columna no existe
    
    # 1. Calcular media y mediana de 'total market value'
    mean_value = df['total market value'].mean()
    median_value = df['total market value'].median()
    print(f"Media del total market value: {mean_value}")
    print(f"Mediana del total market value: {median_value}\n")
    
    # 2. Calcular cuartiles e IQR
    q1 = df['total market value'].quantile(0.25)
    q2 = df['total market value'].quantile(0.50)
    q3 = df['total market value'].quantile(0.75)
    
    iqr = q3 - q1
    print(f"Q1 (25%): {q1}")
    print(f"Q3 (75%): {q3}")
    print(f"Q2 (50%):{q2}")
    print(f"IQR (Rango Intercuartílico): {iqr}\n")

    # Desviación estándar para cada año (usando el dataframe ya cargado)
    std_tmv = df['total market value'].std()
    print(f"Desviación estándar para el Año {year}: {std_tmv}\n")
    
    # 3. Boxplot
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, y='total market value')
    plt.title(f'Boxplot para el Año {year}')
    plt.show()
    
    # 4. Análisis de distribución KDE
    plt.figure(figsize=(10, 6))
    sns.kdeplot(df['total market value'], fill=True)
    plt.title(f'Distribución KDE para el Año {year}')
    plt.xlabel('Total Market Value')
    plt.ylabel('Densidad')
    plt.show()

    # 

# Iterar sobre cada hoja en el diccionario limpio
for sheet_name, df in cleaned_sheets_dict.items():
    # Imprimir las columnas para verificar su existencia
    print(f"Columnas en la hoja {sheet_name}: {df.columns}")
    
    # Extraer el año del nombre de la hoja
    year = sheet_name.split()[-1]
    
    # Llamar a la función de análisis
    analyze_sheet(df, year)

 








In [54]:
# Filtrar por 'Atlético Nacional'
Club = cleaned_sheets_dict['Año_2009'][cleaned_sheets_dict['Año_2009']['club'] == 'CD Real Cartagena']
(Club.head(1))






,club,squad,age,foreigners,market value,total market value
15,CD Real Cartagena,35,22.1,0,53000.0,2100000.0


In [41]:
Atlético_nacional = cleaned_sheets_dict['Año_2009']
(Atlético_nacional.head(20))

,club,squad,age,foreigners,market value,total market value
0,Deportivo Cali,43,22.2,7,148000.0,6380000.0
1,Atlético Nacional,37,23.6,6,161000.0,5950000.0
2,Junior FC,35,25.5,3,163000.0,5700000.0
3,Millonarios FC,38,23.4,5,92000.0,3500000.0
4,Once Caldas,37,23.8,3,92000.0,3400000.0
5,Deportivo Pereira,45,23.1,4,67000.0,3000000.0
6,Envigado FC,38,22.6,1,62000.0,2350000.0
7,Cúcuta Deportivo,40,24.1,3,53000.0,2100000.0
8,CD América de Cali,55,23.9,7,36000.0,2000000.0
9,Independiente Medellín,47,21.0,6,27000.0,1250000.0
